## Capstone Project - Data Cleaning Notebook

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

#### Directory/File Structure

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.23.4
Running numpy version: 1.14.2
Running sklearn version: 0.20.2


In [4]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/notebooks'

In [5]:
os.chdir('../data')

In [6]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/data'

In [7]:
sorted(os.listdir())

['.DS_Store',
 'Capstone - Complication list - complete.xlsx',
 'Capstone - STS risk factor list.xlsx',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'capstone_STS_risk_factor_features.xlsx',
 'capstone_cleaned_data.csv',
 'capstone_data-version-2.xlsx',
 'capstone_data.xlsx',
 'capstone_data_binarized_outcome.xlsx',
 'capstone_data_filled_in_complication_data.xlsx']

#### Loading Dataset
- takes about 90 seconds

In [8]:
raw_data = pd.read_excel('capstone_data_binarized_outcome.xlsx')

### Exploring the Dataset

In [9]:
raw_data.head()

,recordId,age,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,raceother,ethnicity,surgdt,dischdt,heightcm,weightkg,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,TobaccoUse,chrlungd,ChrLungDType,hmo2,bdtx,slpapn,ivdrugab,alcohol,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,cvd,cva,cvawhen,cvdtia,cvdcarsten,cvdstenrt,cvdstenlft,cvdpcarsurg,RFHemoglobin,hct,creatlst,totalbumin,a1clvl,hitanti,inr,meldscr,cigsmoker,cigsmokercurr,cvdcoma,cvdrind,cvdninvas,prcvint,prcab,prvalve,prvalveproc1,PrValveProc2,PrValveProc3,PrValveProc4,POC,POCInt1,POCInt2,POCInt3,POCInt4,POCInt5,poarr,poco,CardSympTimeOfAdm,CardSympTimeOfSurg,anginalclass,chf,classnyh,priorhf,carshock,resusc,Arrhythmia,ArrhythPPaced,ArrhythVV,ArrhythAFlutter,ArrhythAFib,ArrhythAtrFib,ArrhythAFibDur,arrhythwhen,arrhyafib,arrhyafibty,medacei48,medasa,medgp,medgpmn,medacoag,medacmn,medaplt5days,medcoum,MedCoum5Days,MedCoum5Dis,MedXaInhibitors,MedXa5Days,MedNOAC5Days,MedNOACDisc,MedThrombinIn,MedThromIn5Days,MedThromInDisc,medthrom,medinotr,medlipid,medster,numdisv,PctStenLMain,SyntaxScrKnown,SyntaxScr,hdefd,hdef,DimAvail,lvsd,lvedd,pasysmeas,pasys,vdaort,vdstena,AoHemoDatAvail,VDAoVA,vdgrada,VDAoEt1,VDAoEt2,VDAoEt3,VDAoEt4,VDAoEt5,VDAoPrimEt,VDAoSievers,vdinsufm,vdstenm,vdmva,vdgradm,VDMiEt1,VDMiEt2,VDMiEt3,VDMiPrimEt,VDMiLes1,VDMiLes2,VDMiLes3,VDMiPrimLes,vdinsuft,ADPres,ADLocRoot,ADLocAsc,ADLocArch,ADLocDesThor,ADLocThora,ADLesTAneur,ADLesTCoarcNar,ADLesTRup,ADLesTPseudo,ADLesTPenUlcer,ADLesTIntraHema,ADLesTDis,ADLesTDisTmg,ADLesTDisTy,ADEt1,ADEt2,ADEt3,hdefmeth,vdaoet,vdendab,vdcongent,vdprimao,vdlvoutob,vdaorttumor,vdmitpmr,vdmitet,vdmitdegloc,vdmitandegdis,vdmitisty,vdmittumor,hdpad,hdpamean,incidenc,status,UrgEmergRsn,opapp,robotic,RobotTim,opcab,opvalve,vadproc,opocard,oponcard,orentrydt,orexitdt,GenAnes,ProcSed,Intubate,TempMeas,lwsttemp,LwstTempSrc,lwsthct,HighIntraGlu,cpbutil,cpbcmb,cpbcmbr,canartstfem,canartstaort,canartstax,CanArtStInn,canartstoth,perfustm,circarr,dhcatm,cperfutil,cperftime,cperftyp,TotCircArrTm,aortoccl,xclamptm,cplegiadeliv,cplegiatype,ceroxused,concalc,asmtascaa,AsmtAoDxMeth,asmtaodx,asmtapln,ibldprodref,ibldprod,ibdrbcu,ibdffpu,ibdplatu,ibdcryou,IntraClotFact,IntraopProComCon,imedeaca,imedtran,inoptee,prepar,PRepEF,PPEFMeas,PPEF,CombCardPCI,CombProcs,CombProcsStatus,CombProcsPCI,CombProcsStentTy,PPPlanedPCI,ValExpPos,ValExpTyp,ValExpDev,valexp2,valexppos2,ValExpTyp2,valexpdev2,urgntrsn,emergrsn,unplproc,unplav,unplmv,unplao,unplvad,unploth,prerso2lft,prerso2rt,cumulsatlft,cumulsatrt,cofirstind,ibdfactorvii,vad,imedaprot,imedaprotd,imeddesmo,SIStartT,sistopt,afibproc,IABP,iabpwhen,iabpind,inother,opaortic,opmitral,CABHybrPCI,vsavpr,vstcv,vstcvr,VSAVSurgRep,VSAVSurgType,VSAVSurgBioT,VSAVRoot,VSAVRootOReimp,VSAVRootOReimpTy,VSAVRepBioTy,vsmv,vsmvpr,VSTCVMit,MitralImplant,vsmiim,VSTV,VSTrRepair,vsaoimty,cathbasassist,CathBasAssistTy,cathbasassistwhen,cathbasassistind,ecmo,ecmowhen,ecmoind,CompMAD,CompMAD1,CompMAD2,CompMAD3,OCarASDPFO,OCarASDSec,OCarAAProc,OCarAAMeth,OCarAAModel,OCarAAUDI,ocaracd,ocarlva,ocarsvr,ocarvsd,AortProcRoot,AortProcAsc,AortProcHemi,AortProcTotArch,AortProcDesProx,SynthGftEleph,ocarasd,ocarasdty,ocarafibsur,ocarafibsurloc,ocarafibsurlaa,EndovasTAVR,mt30stat,mt30statmeth,mtdate,mtcause,mtopd,mtdcstat,mortalty,mtlocatn,disloctn,dcasa,DCOthAntiplat,dcdirthromin,dccoum,DCFactorXa,DCNovOrAnti,DCOthAnticoag,PostOpPeakGlu,postcreat,PostopHemoglobin,PostopHct,reintub,PostopIntub,popttech,popefd,SurSInf,complics,coprebld,coprebldtim,copregft,CReintMI,CReintMIVes,CReintMIIntTy,CAortReint,CAortReintTy,copreoth,coprenon,csepsis,csepsispbc,cnstrokp,cnstrokttia,cncomaenceph,CNEnceph,cncoma,cnparal,cnparesis,CNParesisTy,cpvntlng,cppneum,cvte,pulmemb,dvt,crenfail,crendial,dialdur,DialStat,cultrafil,cotarrst,CVaAoDisTy,cotcoag,cottamp,cotgi,COtLiver,cotmsf,cotafib,cotother,Readmit,ReadmitDt,readmrsn,cnstroktrind,CNStrokT,drgnum,BldRBC,predstro,predrenf,strokeBin
0,1,54,1.0,1.0,2.0,

In [10]:
raw_data.tail()

,recordId,age,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,raceother,ethnicity,surgdt,dischdt,heightcm,weightkg,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,TobaccoUse,chrlungd,ChrLungDType,hmo2,bdtx,slpapn,ivdrugab,alcohol,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,cvd,cva,cvawhen,cvdtia,cvdcarsten,cvdstenrt,cvdstenlft,cvdpcarsurg,RFHemoglobin,hct,creatlst,totalbumin,a1clvl,hitanti,inr,meldscr,cigsmoker,cigsmokercurr,cvdcoma,cvdrind,cvdninvas,prcvint,prcab,prvalve,prvalveproc1,PrValveProc2,PrValveProc3,PrValveProc4,POC,POCInt1,POCInt2,POCInt3,POCInt4,POCInt5,poarr,poco,CardSympTimeOfAdm,CardSympTimeOfSurg,anginalclass,chf,classnyh,priorhf,carshock,resusc,Arrhythmia,ArrhythPPaced,ArrhythVV,ArrhythAFlutter,ArrhythAFib,ArrhythAtrFib,ArrhythAFibDur,arrhythwhen,arrhyafib,arrhyafibty,medacei48,medasa,medgp,medgpmn,medacoag,medacmn,medaplt5days,medcoum,MedCoum5Days,MedCoum5Dis,MedXaInhibitors,MedXa5Days,MedNOAC5Days,MedNOACDisc,MedThrombinIn,MedThromIn5Days,MedThromInDisc,medthrom,medinotr,medlipid,medster,numdisv,PctStenLMain,SyntaxScrKnown,SyntaxScr,hdefd,hdef,DimAvail,lvsd,lvedd,pasysmeas,pasys,vdaort,vdstena,AoHemoDatAvail,VDAoVA,vdgrada,VDAoEt1,VDAoEt2,VDAoEt3,VDAoEt4,VDAoEt5,VDAoPrimEt,VDAoSievers,vdinsufm,vdstenm,vdmva,vdgradm,VDMiEt1,VDMiEt2,VDMiEt3,VDMiPrimEt,VDMiLes1,VDMiLes2,VDMiLes3,VDMiPrimLes,vdinsuft,ADPres,ADLocRoot,ADLocAsc,ADLocArch,ADLocDesThor,ADLocThora,ADLesTAneur,ADLesTCoarcNar,ADLesTRup,ADLesTPseudo,ADLesTPenUlcer,ADLesTIntraHema,ADLesTDis,ADLesTDisTmg,ADLesTDisTy,ADEt1,ADEt2,ADEt3,hdefmeth,vdaoet,vdendab,vdcongent,vdprimao,vdlvoutob,vdaorttumor,vdmitpmr,vdmitet,vdmitdegloc,vdmitandegdis,vdmitisty,vdmittumor,hdpad,hdpamean,incidenc,status,UrgEmergRsn,opapp,robotic,RobotTim,opcab,opvalve,vadproc,opocard,oponcard,orentrydt,orexitdt,GenAnes,ProcSed,Intubate,TempMeas,lwsttemp,LwstTempSrc,lwsthct,HighIntraGlu,cpbutil,cpbcmb,cpbcmbr,canartstfem,canartstaort,canartstax,CanArtStInn,canartstoth,perfustm,circarr,dhcatm,cperfutil,cperftime,cperftyp,TotCircArrTm,aortoccl,xclamptm,cplegiadeliv,cplegiatype,ceroxused,concalc,asmtascaa,AsmtAoDxMeth,asmtaodx,asmtapln,ibldprodref,ibldprod,ibdrbcu,ibdffpu,ibdplatu,ibdcryou,IntraClotFact,IntraopProComCon,imedeaca,imedtran,inoptee,prepar,PRepEF,PPEFMeas,PPEF,CombCardPCI,CombProcs,CombProcsStatus,CombProcsPCI,CombProcsStentTy,PPPlanedPCI,ValExpPos,ValExpTyp,ValExpDev,valexp2,valexppos2,ValExpTyp2,valexpdev2,urgntrsn,emergrsn,unplproc,unplav,unplmv,unplao,unplvad,unploth,prerso2lft,prerso2rt,cumulsatlft,cumulsatrt,cofirstind,ibdfactorvii,vad,imedaprot,imedaprotd,imeddesmo,SIStartT,sistopt,afibproc,IABP,iabpwhen,iabpind,inother,opaortic,opmitral,CABHybrPCI,vsavpr,vstcv,vstcvr,VSAVSurgRep,VSAVSurgType,VSAVSurgBioT,VSAVRoot,VSAVRootOReimp,VSAVRootOReimpTy,VSAVRepBioTy,vsmv,vsmvpr,VSTCVMit,MitralImplant,vsmiim,VSTV,VSTrRepair,vsaoimty,cathbasassist,CathBasAssistTy,cathbasassistwhen,cathbasassistind,ecmo,ecmowhen,ecmoind,CompMAD,CompMAD1,CompMAD2,CompMAD3,OCarASDPFO,OCarASDSec,OCarAAProc,OCarAAMeth,OCarAAModel,OCarAAUDI,ocaracd,ocarlva,ocarsvr,ocarvsd,AortProcRoot,AortProcAsc,AortProcHemi,AortProcTotArch,AortProcDesProx,SynthGftEleph,ocarasd,ocarasdty,ocarafibsur,ocarafibsurloc,ocarafibsurlaa,EndovasTAVR,mt30stat,mt30statmeth,mtdate,mtcause,mtopd,mtdcstat,mortalty,mtlocatn,disloctn,dcasa,DCOthAntiplat,dcdirthromin,dccoum,DCFactorXa,DCNovOrAnti,DCOthAnticoag,PostOpPeakGlu,postcreat,PostopHemoglobin,PostopHct,reintub,PostopIntub,popttech,popefd,SurSInf,complics,coprebld,coprebldtim,copregft,CReintMI,CReintMIVes,CReintMIIntTy,CAortReint,CAortReintTy,copreoth,coprenon,csepsis,csepsispbc,cnstrokp,cnstrokttia,cncomaenceph,CNEnceph,cncoma,cnparal,cnparesis,CNParesisTy,cpvntlng,cppneum,cvte,pulmemb,dvt,crenfail,crendial,dialdur,DialStat,cultrafil,cotarrst,CVaAoDisTy,cotcoag,cottamp,cotgi,COtLiver,cotmsf,cotafib,cotother,Readmit,ReadmitDt,readmrsn,cnstroktrind,CNStrokT,drgnum,BldRBC,predstro,predrenf,strokeBin
42741,42742,62,1.0,

In [11]:
raw_data.shape

(42746, 409)

#### Examining Datatypes

In [12]:
data_types = pd.DataFrame(raw_data.dtypes,
                         columns = ['data_type'])

In [13]:
data_types.head()

,data_type
recordId,int64
age,int64
gender,float64
racecaucasian,float64
raceblack,float64


In [14]:
# data types in data set
data_types['data_type'].unique()

array([dtype('int64'), dtype('float64'), dtype('<M8[ns]')], dtype=object)

In [15]:
# data types in data set
data_types['data_type'].value_counts()

float64           354
int64              49
datetime64[ns]      6
Name: data_type, dtype: int64

In [16]:
# examine features by data type - `datetime64[ns]`
data_types.loc[data_types['data_type'] == 'datetime64[ns]']

,data_type
surgdt,datetime64[ns]
dischdt,datetime64[ns]
orentrydt,datetime64[ns]
orexitdt,datetime64[ns]
mtdate,datetime64[ns]
ReadmitDt,datetime64[ns]


In [17]:
# examine features by data type - `int64` -- take off head() to see full list
data_types.loc[data_types['data_type'] == 'int64'].head()

,data_type
recordId,int64
age,int64
hypertn,int64
immsupp,int64
medgp,int64


#### Looking at the Number of Null Values per Feature

In [18]:
col_names = []
num_nulls = []

for column in raw_data.columns:
    col_names.append(column)
    num_nulls.append(raw_data[column].isnull().sum())

In [19]:
print(len(col_names))
print(len(num_nulls))

409
409


In [20]:
# creating a new data frame for easier analysis

nulls_df = pd.DataFrame(list(zip(col_names, num_nulls)),
                       columns = ['feature', 'number_of_nulls'])

In [21]:
nulls_df.head()

,feature,number_of_nulls
0,recordId,0
1,age,0
2,gender,3
3,racecaucasian,447
4,raceblack,448


In [22]:
nulls_df.tail()

,feature,number_of_nulls
404,drgnum,42746
405,BldRBC,42746
406,predstro,3
407,predrenf,1054
408,strokeBin,0


In [23]:
# sorting by features with the most number of nulls
nulls_df_sorted = nulls_df.sort_values(by=['number_of_nulls'],
                                       ascending=False)

In [24]:
nulls_df_sorted.head()

,feature,number_of_nulls
194,GenAnes,42746
196,Intubate,42746
55,cvdcoma,42746
271,imedaprotd,42746
272,imeddesmo,42746


#### - From a cleaning perspective, should we get rid of features with all nulls??

In [25]:
nulls_df_sorted[nulls_df_sorted['number_of_nulls'] == 42746]

,feature,number_of_nulls
194,GenAnes,42746
196,Intubate,42746
55,cvdcoma,42746
271,imedaprotd,42746
272,imeddesmo,42746
273,SIStartT,42746
274,sistopt,42746
279,inother,42746
280,opaortic,42746
285,vstcvr,42746


In [26]:
# there is a significant number of features with all rows null
nulls_df_sorted[nulls_df_sorted['number_of_nulls'] == 42746].shape[0]

56

#### If these features are not important
- export series to a list and then use to mask main dataset to get eliminate those features - see code below

In [27]:
# list(nulls_df_sorted[nulls_df_sorted['number_of_nulls'] == 42746]['feature'])

#### Looking at the features with no or few nulls

In [28]:
nulls_df_sorted[nulls_df_sorted['number_of_nulls'] < 20]

,feature,number_of_nulls
236,imedtran,15
235,imedeaca,13
191,oponcard,13
108,medthrom,12
305,ecmo,12
22,chrlungd,11
301,cathbasassist,10
189,vadproc,10
334,mt30stat,8
98,medcoum,8


In [29]:
nulls_df_sorted[nulls_df_sorted['number_of_nulls'] < 20].shape[0]

109

#### Looing at the Unique Number of Values per Feature
- using the same code pattern as when looking at null values per feature above

In [30]:
col_names = []
num_unique = []

for column in raw_data.columns:
    col_names.append(column)
    num_unique.append(raw_data[column].nunique())

In [31]:
print(len(col_names))
print(len(num_unique))

409
409


In [32]:
nuniques_df = pd.DataFrame(list(zip(col_names, num_unique)),
                           columns = ['feature', 'nuniques']).sort_values(by=['nuniques'],
                                                                          ascending=False)

In [33]:
# Top 50 Features in terms of unique values - many are self-explanatory

nuniques_df.head(50)

,feature,nuniques
0,recordId,42746
11,dischdt,2032
192,orentrydt,1957
193,orexitdt,1957
10,surgdt,1955
13,weightkg,1223
52,meldscr,1059
336,mtdate,854
400,ReadmitDt,835
122,pasys,602


#### Should we eliminate features with too many categories - increase dimensionality of dataset??
- of the list above, can we distinguish between numerical and categorical features??

In [34]:
# this will yield some of the features with all nulls
nuniques_df.tail()

,feature,nuniques
57,cvdninvas,0
56,cvdrind,0
55,cvdcoma,0
299,VSTrRepair,0
103,MedNOAC5Days,0


#### This should give you most of the categorical variables

In [35]:
cat_features_df = nuniques_df[(nuniques_df['nuniques'] > 0) & (nuniques_df['nuniques'] <= 25)]

- pandas syntax note on filtering on multiple conditions - each condition must be in parentheses

In [36]:
cat_features_df.shape

(308, 2)

In [37]:
cat_features_df.head(10)

,feature,nuniques
183,UrgEmergRsn,25
128,VDAoEt1,25
129,VDAoEt2,23
229,ibdrbcu,22
230,ibdffpu,18
231,ibdplatu,18
67,POCInt2,17
184,opapp,16
144,VDMiLes2,15
143,VDMiLes1,15


#### Number of Binary Categorical Features

In [38]:
nuniques_df[nuniques_df['nuniques'] == 2].shape[0]

134

- if you want to examine further - assign above `dataframe` to a variable name

### Testing Binarization Process

#### Goal is an efficient process to binarize variables

- let's pick 3 features with different number of categories (3, 5, 7)

In [39]:
nuniques_df[nuniques_df['nuniques'] == 3].head()

,feature,nuniques
343,dcasa,3
202,cpbutil,3
58,prcvint,3
175,vdmitdegloc,3
331,ocarafibsurloc,3


In [40]:
nuniques_df[nuniques_df['nuniques'] == 5].head()

,feature,nuniques
304,cathbasassistind,5
372,cnstrokp,5
181,incidenc,5
308,CompMAD,5
131,VDAoEt4,5


In [41]:
nuniques_df[nuniques_df['nuniques'] == 7].head()

,feature,nuniques
342,disloctn,7
74,CardSympTimeOfSurg,7
73,CardSympTimeOfAdm,7
249,ValExpTyp,7
15,diabctrl,7


- selecting our test features
- `dcasa`, `CompMAD`, `ValExpTyp`

- let's look the unique values for each feature

In [42]:
print(raw_data['dcasa'].unique())
print(len(raw_data['dcasa']))

[ 1.  3. nan  2.]
42746


In [43]:
print(raw_data['CompMAD'].unique())
print(len(raw_data['CompMAD']))

[nan  1.  3.  2.  4.  5.]
42746


In [44]:
print(raw_data['ValExpTyp'].unique())
print(len(raw_data['ValExpTyp']))

[nan  3.  4.  2.  1.  6.  7.  9.]
42746


Before recoding we need to get rid of the NaNs - make them 0??

`pd.get_dummies` assigns NaNs `0`

- let's do this step by step and then put into a function

In [45]:
test_bin_features = ['dcasa',
                     'CompMAD',
                     'ValExpTyp']

In [46]:
test_bin_df = raw_data[test_bin_features]

In [47]:
test_bin_df.head()

,dcasa,CompMAD,ValExpTyp
0,1.0,NaN,NaN
1,1.0,NaN,NaN
2,1.0,NaN,NaN
3,1.0,NaN,NaN
4,1.0,NaN,NaN


In [48]:
test_bin_df.shape

(42746, 3)

In [49]:
test_bin_dummy1 = pd.get_dummies(test_bin_df['dcasa'])

In [50]:
test_bin_dummy1.head()

,1.0,2.0,3.0
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [51]:
test_bin_dummy1.shape

(42746, 3)

In [52]:
test_bin_dummy1.dtypes

1.0    uint8
2.0    uint8
3.0    uint8
dtype: object

In [53]:
# this should equal the number of nulls in `dcasa`

test_bin_dummy1[(test_bin_dummy1[1.0] == 0) & (test_bin_dummy1[2.0] == 0) & (test_bin_dummy1[3.0] == 0)].shape

(736, 3)

In [54]:
# confirming

raw_data['dcasa'].isnull().sum()

736

#### Works!

#### Can apply `pd.get_dummies` to `test_bin_df`
- interested in how it handles column headings

In [55]:
test_bin_dummies = pd.get_dummies(test_bin_df)

In [56]:
test_bin_dummies.head()

,dcasa,CompMAD,ValExpTyp
0,1.0,NaN,NaN
1,1.0,NaN,NaN
2,1.0,NaN,NaN
3,1.0,NaN,NaN
4,1.0,NaN,NaN


- This does not work because `pd.get_dummies` ignores numeric columns and only works on strings
- the interesting part is that it works for single column
- testing converting whole dataframe to string via `df.astype(str)` and then apply `pd.get_dummies`

In [57]:
test_bin_df.dtypes

dcasa        float64
CompMAD      float64
ValExpTyp    float64
dtype: object

In [58]:
test_bin_df_str = test_bin_df.astype(str)

In [59]:
test_bin_df_str.dtypes

dcasa        object
CompMAD      object
ValExpTyp    object
dtype: object

In [60]:
print (test_bin_df.shape)
print (test_bin_df_str.shape)

(42746, 3)
(42746, 3)


#### Now trying `pd.get_dummies` on `test_bin_df_str`

In [61]:
test_bin_dummies = pd.get_dummies(test_bin_df_str)

In [62]:
test_bin_dummies.head()

,dcasa_1.0,dcasa_2.0,dcasa_3.0,dcasa_nan,CompMAD_1.0,CompMAD_2.0,CompMAD_3.0,CompMAD_4.0,CompMAD_5.0,CompMAD_nan,ValExpTyp_1.0,ValExpTyp_2.0,ValExpTyp_3.0,ValExpTyp_4.0,ValExpTyp_6.0,ValExpTyp_7.0,ValExpTyp_9.0,ValExpTyp_nan
0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [63]:
test_bin_dummies.shape

(42746, 18)

In [64]:
test_bin_dummies.dtypes

dcasa_1.0        uint8
dcasa_2.0        uint8
dcasa_3.0        uint8
dcasa_nan        uint8
CompMAD_1.0      uint8
CompMAD_2.0      uint8
CompMAD_3.0      uint8
CompMAD_4.0      uint8
CompMAD_5.0      uint8
CompMAD_nan      uint8
ValExpTyp_1.0    uint8
ValExpTyp_2.0    uint8
ValExpTyp_3.0    uint8
ValExpTyp_4.0    uint8
ValExpTyp_6.0    uint8
ValExpTyp_7.0    uint8
ValExpTyp_9.0    uint8
ValExpTyp_nan    uint8
dtype: object

#### Note the `uint8` `datatype` assigned by `pd.get_dummies` in case there is a problem later with some of the algorithms which are fussy in terms of `datatype`

### Close, but code above it is coding the `NaNs` into a separate column

#### Want `dcasa` `NaNs` to be coded as `0`, `0`, `0` for `dcasa_1.0`, `dcasa_2.0`, `dcasa_3.0`

- theoretically could just select all the `_nan` columns and then delete

- feels like a hack, but it should work

In [65]:
drop_col = [col for col in test_bin_dummies if col.endswith('_nan')]

In [66]:
drop_col

['dcasa_nan', 'CompMAD_nan', 'ValExpTyp_nan']

#### Now dropping the `_nan` columns using `df.drop()`

In [67]:
test_bin_dummies_final = test_bin_dummies.drop(drop_col, axis=1)

In [68]:
test_bin_dummies_final.head()

,dcasa_1.0,dcasa_2.0,dcasa_3.0,CompMAD_1.0,CompMAD_2.0,CompMAD_3.0,CompMAD_4.0,CompMAD_5.0,ValExpTyp_1.0,ValExpTyp_2.0,ValExpTyp_3.0,ValExpTyp_4.0,ValExpTyp_6.0,ValExpTyp_7.0,ValExpTyp_9.0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
test_bin_dummies_final.shape

(42746, 15)

### Works! - Now Need to Put Into a Function

#### Function Pseudocode

- select columns from dataset you want to `binarize` assign them to a `list`
- convert resulting `dataframe` `dtype` to `str` using `astype`
- `drop` `_nan` columns
- return `dataframe`

### `categorical_to_numeric` Function

In [70]:
def categorial_to_numeric(cols_to_binarize, data):
    
    # select columns to binarize
    categorical_features = data[cols_to_binarize]
    
    # convert categorical_features dataframe datatype to str
    categorical_features = categorical_features.astype(str)
    
    # use pd.get_dummies to generate dummy variables for each category level
    dummies = pd.get_dummies(categorical_features)
    
    # identifying nan columns to drop
    drop_cols = [col for col in dummies if col.endswith('_nan')]
    
    # dropping the _nan columns
    final_df = dummies.drop(drop_cols, axis=1)
    
    # return the final dataframe
    return final_df

- validating function

In [71]:
test_bin_features

['dcasa', 'CompMAD', 'ValExpTyp']

In [72]:
test_cat_num_function = categorial_to_numeric(test_bin_features, raw_data)

In [73]:
test_cat_num_function.head()

,dcasa_1.0,dcasa_2.0,dcasa_3.0,CompMAD_1.0,CompMAD_2.0,CompMAD_3.0,CompMAD_4.0,CompMAD_5.0,ValExpTyp_1.0,ValExpTyp_2.0,ValExpTyp_3.0,ValExpTyp_4.0,ValExpTyp_6.0,ValExpTyp_7.0,ValExpTyp_9.0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
test_cat_num_function.shape

(42746, 15)

#### Confirming that function handled `NaNs` correctly

In [75]:
# this should equal the number of nulls in `dcasa`
print(test_cat_num_function[(test_cat_num_function['dcasa_1.0'] == 0) &
                            (test_cat_num_function['dcasa_2.0'] == 0) &
                            (test_cat_num_function['dcasa_3.0'] == 0)].shape[0])

# confirming
print(raw_data['dcasa'].isnull().sum())

736
736


In [76]:
# this should equal the number of nulls in `CompMAD`
print(test_cat_num_function[(test_cat_num_function['CompMAD_1.0'] == 0) &
                            (test_cat_num_function['CompMAD_2.0'] == 0) &
                            (test_cat_num_function['CompMAD_3.0'] == 0) &
                            (test_cat_num_function['CompMAD_4.0'] == 0) &
                            (test_cat_num_function['CompMAD_5.0'] == 0)].shape[0])

# confirming
print(raw_data['CompMAD'].isnull().sum())

23620
23620


In [77]:
# this should equal the number of nulls in `ValExpTyp`
print(test_cat_num_function[(test_cat_num_function['ValExpTyp_1.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_2.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_3.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_4.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_6.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_7.0'] == 0) &
                            (test_cat_num_function['ValExpTyp_9.0'] == 0)].shape[0])

# confirming
print(raw_data['ValExpTyp'].isnull().sum())

42155
42155


#### Testing if you can apply functions to single columns, if needed

In [78]:
test_single_col = ['diabctrl']

In [79]:
test_cat_num_function_single = categorial_to_numeric(test_single_col, raw_data)

In [80]:
test_cat_num_function_single.head()

,diabctrl_1.0,diabctrl_2.0,diabctrl_3.0,diabctrl_4.0,diabctrl_5.0,diabctrl_6.0,diabctrl_7.0
0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0


In [81]:
test_cat_num_function_single.shape

(42746, 7)

In [82]:
# this should equal the number of nulls in `diabctrl`
print(test_cat_num_function[(test_cat_num_function_single['diabctrl_1.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_2.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_3.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_4.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_5.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_6.0'] == 0) &
                            (test_cat_num_function_single['diabctrl_7.0'] == 0)].shape[0])

# confirming
print(raw_data['diabctrl'].isnull().sum())

24791
24791


### `categorical_to_numeric` Function Workflow
- can apply multiple times, creating multiple `dataframes`
- then can combine into a single `dataframe` using `pd.concat`

In [83]:
combined_binarized = pd.concat((test_cat_num_function,
                               test_cat_num_function_single),
                               axis=1) 

# syntax note - df's to be combined via pd.concat have to be contained within parentheses

In [84]:
combined_binarized.head()

,dcasa_1.0,dcasa_2.0,dcasa_3.0,CompMAD_1.0,CompMAD_2.0,CompMAD_3.0,CompMAD_4.0,CompMAD_5.0,ValExpTyp_1.0,ValExpTyp_2.0,ValExpTyp_3.0,ValExpTyp_4.0,ValExpTyp_6.0,ValExpTyp_7.0,ValExpTyp_9.0,diabctrl_1.0,diabctrl_2.0,diabctrl_3.0,diabctrl_4.0,diabctrl_5.0,diabctrl_6.0,diabctrl_7.0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
print(combined_binarized.shape)
print(test_cat_num_function.shape)
print(test_cat_num_function_single.shape)

(42746, 22)
(42746, 15)
(42746, 7)


## Creating Additional Features from `datetime` Features
- here is a list of the `datetime64` features

In [86]:
data_types.loc[data_types['data_type'] == 'datetime64[ns]']

,data_type
surgdt,datetime64[ns]
dischdt,datetime64[ns]
orentrydt,datetime64[ns]
orexitdt,datetime64[ns]
mtdate,datetime64[ns]
ReadmitDt,datetime64[ns]


- Looking into `NaNs` of `datetime` features

In [87]:
print(raw_data['surgdt'].isnull().sum())
print(raw_data['dischdt'].isnull().sum())
print(raw_data['orentrydt'].isnull().sum())
print(raw_data['orexitdt'].isnull().sum())
print(raw_data['mtdate'].isnull().sum())
print(raw_data['ReadmitDt'].isnull().sum())

0
0
0
0
41625
40583


In [88]:
date_cols = ['surgdt',
             'dischdt',
             'orentrydt',
             'orexitdt']

In [89]:
date_features = raw_data[date_cols]

In [90]:
date_features.head()

,surgdt,dischdt,orentrydt,orexitdt
0,2011-07-01,2011-07-06,2011-07-01,2011-07-01
1,2011-07-02,2011-07-09,2011-07-02,2011-07-03
2,2011-07-04,2011-07-12,2011-07-04,2011-07-04
3,2011-07-05,2011-07-09,2011-07-05,2011-07-05
4,2011-07-06,2011-07-10,2011-07-06,2011-07-06


In [91]:
date_features.shape

(42746, 4)

#### Extracting Info from Time Stamp

In [92]:
print (date_features['surgdt'][0])
print (type(date_features['surgdt'][0]))
print (date_features['surgdt'][0].year)
print (date_features['surgdt'][0].month)
print (date_features['surgdt'][0].day)
print (date_features['surgdt'][0].timetuple()) 
print (date_features['surgdt'][0].hour)
print (date_features['surgdt'][0].weekday()) # day of week as an integer, Monday = 0, Sunday = 6

2011-07-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2011
7
1
time.struct_time(tm_year=2011, tm_mon=7, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=182, tm_isdst=-1)
0
4


- unfortunately, it seems that the time of surgery (`.hour`) was not embedded in the timestamp

#### Creating New `datetime` Features

In [95]:
def date_components(data, col_labels):
    '''this function extracts date components from datetime objecr and recenters them
       where appropriate
    '''
    dates_frame = data.apply(lambda x: pd.Series([x.year, 
                                                  x.month,
                                                  x.day,
                                                  x.weekday()]))
                                                  #x.hour]))
    dates_frame.columns = col_labels
        
    return dates_frame

- creating labels for new features

In [96]:
surgdt_col_labels = ['surgdt_year',
                     'surgdt_month',
                     'surgdt_DayOfMonth',
                     'surgdt_DayOfWeek']
                     #'surgdt_hour']

In [97]:
surgdt_features = date_components(date_features['surgdt'], surgdt_col_labels)

In [98]:
surgdt_features.head()

,surgdt_year,surgdt_month,surgdt_DayOfMonth,surgdt_DayOfWeek
0,2011,7,1,4
1,2011,7,2,5
2,2011,7,4,0
3,2011,7,5,1
4,2011,7,6,2


In [99]:
surgdt_features.shape

(42746, 4)

In [100]:
# confirming that no time of day recorded -- confirmed in prior version of function
# surgdt_features['surgdt_hour'].value_counts()

#### May want to convert `surgdt_weekday` to dummies
- first convert numerical weekday to text using a dictionary

In [101]:
surgdt_features['surgdt_DayOfWeek'].value_counts()

0    9246
2    8199
1    8088
4    8025
3    7633
5     991
6     564
Name: surgdt_DayOfWeek, dtype: int64

In [102]:
weekday_dict = {0: "Mon",
                1: "Tues",
                2: "Wed",
                3: "Thurs",
                4: "Fri",
                5: "Sat",
                6: "Sun"}

- now replace numerical values with text using `df.replace({'col_name': replacement_dict})`

In [104]:
surgdt_features_mod = surgdt_features.replace({'surgdt_DayOfWeek': weekday_dict})

In [105]:
surgdt_features_mod.head()

,surgdt_year,surgdt_month,surgdt_DayOfMonth,surgdt_DayOfWeek
0,2011,7,1,Fri
1,2011,7,2,Sat
2,2011,7,4,Mon
3,2011,7,5,Tues
4,2011,7,6,Wed


In [106]:
surgdt_features_mod.dtypes

surgdt_year           int64
surgdt_month          int64
surgdt_DayOfMonth     int64
surgdt_DayOfWeek     object
dtype: object

In [107]:
surgdt_features_mod.shape

(42746, 4)

In [108]:
surgdt_features_mod['surgdt_DayOfWeek'].isnull().sum()

0

- now should be able to apply `pd.get_dummies`

In [109]:
surgdt_features_final = pd.get_dummies(surgdt_features_mod)

In [110]:
surgdt_features_final.head()

,surgdt_year,surgdt_month,surgdt_DayOfMonth,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Wed
0,2011,7,1,1,0,0,0,0,0,0
1,2011,7,2,0,0,1,0,0,0,0
2,2011,7,4,0,1,0,0,0,0,0
3,2011,7,5,0,0,0,0,0,1,0
4,2011,7,6,0,0,0,0,0,0,1


In [111]:
surgdt_features_final.shape

(42746, 10)

#### Alternatively, since it looks like you may have to binarize `year`, `month`, `day` - could use a modified version of `categorical_to_numeric` function

In [112]:
def categorial_to_numeric_dates(date_feature_df):
    
    # date_feature_df - from extracted date components from date feature of interest
    
    # convert categorical_features dataframe datatype to str
    date_feature_df = date_feature_df.astype(str)
    
    # use pd.get_dummies to generate dummy variables for each category level
    dummies = pd.get_dummies(date_feature_df)
    
    # skim the _nan portion of the function because the main date features do not have nulls
    
    # return the final dataframe
    return dummies

- `date_feature_df` for testing the function is `surgdt_features_mod`, but first need to drop the `hour` column

In [114]:
# already excluded `surgdt_hour` above

#surgdt_features_mod = surgdt_features_mod.drop(['surgdt_hour'], axis=1)

In [115]:
surgdt_features_mod.head()

,surgdt_year,surgdt_month,surgdt_DayOfMonth,surgdt_DayOfWeek
0,2011,7,1,Fri
1,2011,7,2,Sat
2,2011,7,4,Mon
3,2011,7,5,Tues
4,2011,7,6,Wed


In [116]:
surgdt_features_mod.shape

(42746, 4)

- need to go back and modify `date_components` function if all `datetime` features lack `hour` information
- then you would not need the last step

#### Testing `categorial_to_numeric_dates` Function

In [117]:
surgdt_features_function = categorial_to_numeric_dates(surgdt_features_mod)

In [118]:
surgdt_features_function.head()

,surgdt_year_2011,surgdt_year_2012,surgdt_year_2013,surgdt_year_2014,surgdt_year_2015,surgdt_year_2016,surgdt_month_1,surgdt_month_10,surgdt_month_11,surgdt_month_12,surgdt_month_2,surgdt_month_3,surgdt_month_4,surgdt_month_5,surgdt_month_6,surgdt_month_7,surgdt_month_8,surgdt_month_9,surgdt_DayOfMonth_1,surgdt_DayOfMonth_10,surgdt_DayOfMonth_11,surgdt_DayOfMonth_12,surgdt_DayOfMonth_13,surgdt_DayOfMonth_14,surgdt_DayOfMonth_15,surgdt_DayOfMonth_16,surgdt_DayOfMonth_17,surgdt_DayOfMonth_18,surgdt_DayOfMonth_19,surgdt_DayOfMonth_2,surgdt_DayOfMonth_20,surgdt_DayOfMonth_21,surgdt_DayOfMonth_22,surgdt_DayOfMonth_23,surgdt_DayOfMonth_24,surgdt_DayOfMonth_25,surgdt_DayOfMonth_26,surgdt_DayOfMonth_27,surgdt_DayOfMonth_28,surgdt_DayOfMonth_29,surgdt_DayOfMonth_3,surgdt_DayOfMonth_30,surgdt_DayOfMonth_31,surgdt_DayOfMonth_4,surgdt_DayOfMonth_5,surgdt_DayOfMonth_6,surgdt_DayOfMonth_7,surgdt_DayOfMonth_8,surgdt_DayOfMonth_9,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Wed
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [120]:
surgdt_features_function.shape # see the significant difference in number of features

(42746, 56)

#### Key Takeaway - will create a lot more features for each `datetime` feature converted - dimensionality concerns

## Pipelines for Preprocessing Numerical Features
- will eventually need to select and scale certain features like `age` and `weightkg`
- building `sklearn` `pipelines` for doing this efficiently

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [121]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

#### Creating a Function to Replace Null Values with Feature `median`
- let's look at `weightkg` and `heightcm` `NaNs`

In [122]:
raw_data['weightkg'][raw_data['weightkg'].isnull()]

8710    NaN
22182   NaN
29600   NaN
Name: weightkg, dtype: float64

In [123]:
raw_data['heightcm'][raw_data['heightcm'].isnull()]

22182   NaN
29600   NaN
Name: heightcm, dtype: float64

- 22182 and 29600 overlap as not recording height and weight - should we just drop these 2 rows???
- can do this by: `df.drop(df.index[[22182, 29600]])` - note double brackets around row numbers

In [124]:
print (raw_data['weightkg'].median())
print (raw_data['heightcm'].median())

87.300003
172.7


In [125]:
# to extract series name
raw_data['weightkg'].name

'weightkg'

### `fill_nulls_median` Function

In [127]:
def fill_nulls_median(data):
    cleaned = data.fillna(data.median())
    return cleaned

#### Could also create a function that fills in `NaNs` with `mean()`

#### Testing `fill_nulls_median` Function

In [128]:
test_fill_nulls_median = fill_nulls_median(raw_data['weightkg'])

In [129]:
type(test_fill_nulls_median)

pandas.core.series.Series

In [130]:
test_fill_nulls_median.isnull().sum()

0

In [131]:
test_fill_nulls_median[8710]

87.300003

#### Use `FunctionTransformer` to incorporate custom functions in a `Pipeline`

### `numerical_features_pipeline` - Stage 1
- select features
- replace `NaNs` with feature `median` values

In [132]:
numerical_features_pipeline = Pipeline(steps=[
        ('select_features', FeatureSelector(['age', 'heightcm', 'weightkg'])),
        ('impute_nulls', FunctionTransformer(fill_nulls_median, validate=False))
        #('min_max_scale', MinMaxScaler()) - last step will be a scaling function
    ])

In [133]:
test_pipe = numerical_features_pipeline.fit_transform(raw_data)

In [134]:
test_pipe.head()

,age,heightcm,weightkg
0,54,180.00000,117.0
1,65,175.30000,79.4
2,83,162.60001,102.1
3,59,160.00000,127.5
4,72,160.00000,64.0


- seeing if nulls removed

In [135]:
test_pipe['weightkg'].isnull().sum()

0

In [136]:
test_pipe['heightcm'].isnull().sum()

0

- seeing if null values replaced with medians correctly

In [137]:
test_pipe.iloc[[8710, 22182, 29600]]

,age,heightcm,weightkg
8710,62,170.2,87.3
22182,65,172.7,87.3
29600,63,172.7,87.3


In [138]:
test_pipe.shape

(42746, 3)

### Now adding `StandardScaler()` to `Pipeline`

In [139]:
numerical_features_pipeline_final = Pipeline(steps=[
        ('select_features', FeatureSelector(['age', 'heightcm', 'weightkg'])),
        ('impute_nulls', FunctionTransformer(fill_nulls_median, validate=False)),
        ('standard_scale', StandardScaler())
    ])

### NOTE: Need to apply separate scaling to `X_train` and `X_test` sets
#### - `fit_transform(X_train)` - `fit_transform` on training datas
#### - `transform(X_test)` - `transform` only on test/development data
#### - do this to prevent training data from leaking into test data

In [140]:
test_pipe_final = numerical_features_pipeline_final.fit_transform(raw_data)

/Users/nate_velarde/anaconda/envs/py3jp/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/nate_velarde/anaconda/envs/py3jp/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [141]:
type(test_pipe_final)

numpy.ndarray

- NOTE: `Pipeline` returns a `numpy.ndarray` since `StandardScaler()` is the last step
- Final step in workflow would be to convert to `dataframe`

In [142]:
test_pipe_final_df = pd.DataFrame(test_pipe_final,
                                  columns=['age', 'heightcm', 'weightkg'])

In [143]:
test_pipe_final_df.head()

,age,heightcm,weightkg
0,-1.11659,0.79391,1.34327
1,-0.10871,0.35690,-0.49369
2,1.54054,-0.82397,0.61532
3,-0.65846,-1.06572,1.85624
4,0.53266,-1.06572,-1.24606


In [144]:
test_pipe_final_df.shape

(42746, 3)

### Once you have selected and created the features you want for the feature matrix (`X`)
#### - need to stich together the resulting separate `dataframes` - use `pd.concat()` per code snippet below